# GDC Query Evaluation Framework

This notebook evaluates 30 queries across three complexity levels against the Genomic Data Commons (GDC) API:
- **Basic Discovery (Low Complexity)**: 10 queries (EV-L01 to EV-L10)
- **Entity Filtering (Medium Complexity)**: 10 queries (EV-M01 to EV-M10)  
- **Complex Cohorts (High Complexity)**: 10 queries (EV-H01 to EV-H10)

In [3]:
# Import Required Libraries

import requests
import json
import time

In [4]:

# GDC API Configuration
GDC_API_BASE = "https://api.gdc.cancer.gov"
results = {}

In [5]:
# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def graphql_query(query, variables=None):
    """Execute GraphQL query against GDC"""
    url = f"{GDC_API_BASE}/v0/graphql"
    headers = {"Content-Type": "application/json"}
    payload = {"query": query}
    if variables:
        payload["variables"] = variables

    response = requests.post(url, json=payload, headers=headers)

    # Better error handling
    if response.status_code != 200:
        print(f"❌ GraphQL Error: {response.status_code}")
        print(f"Response: {response.text}")
        return None

    result = response.json()
    if "errors" in result:
        print(f"❌ GraphQL Errors: {json.dumps(result['errors'], indent=2)}")
        return None

    return result


def rest_query(endpoint, params=None):
    """Execute REST API query against GDC"""
    url = f"{GDC_API_BASE}/{endpoint}"
    headers = {"Content-Type": "application/json"}

    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status()
    return response.json()

In [4]:
# EV-L01: In the GDC database, list all available program names
# Direct entity list (program names)
def eval_L01():
    start = time.time()
    try:
        # Get programs through projects endpoint since programs endpoint doesn't exist
        result = rest_query("projects", {
            "size": "2000",
            "fields": "program.name"
        })
        
        # Extract unique program names
        programs = set()
        for project in result["data"]["hits"]:
            program_info = project.get("program", {})
            if isinstance(program_info, dict) and "name" in program_info:
                programs.add(program_info["name"])
            elif isinstance(program_info, list):
                for prog in program_info:
                    if isinstance(prog, dict) and "name" in prog:
                        programs.add(prog["name"])
        
        programs_list = sorted(list(programs))
        count = len(programs_list)
        
        print(f"✅ EV-L01: Found {count} programs")
        print(f"Programs: {', '.join(programs_list)}")
        
        results["EV-L01"] = {
            "status": "success",
            "result": f"{count} programs",
            "data": programs_list,
            "time": time.time() - start,
        }
    except Exception as e:
        print(f"❌ EV-L01 Failed: {e}")
        results["EV-L01"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_L01()

✅ EV-L01: Found 23 programs
Programs: APOLLO, BEATAML1.0, CCDI, CDDP_EAGLE, CGCI, CMI, CPTAC, CTSP, EXCEPTIONAL_RESPONDERS, FM, HCMI, MATCH, MMRF, MP2PRT, NCICCR, OHSU, ORGANOID, REBC, TARGET, TCGA, TRIO, VAREPOP, WCDT


In [5]:
# EV-L02: In the GDC database, count the total number of projects
# Simple count (total projects)
def eval_L02():
    start = time.time()
    try:
        result = rest_query("projects", {"size": "0"})
        count = result["data"]["pagination"]["total"]
        
        print(f"✅ EV-L02: Found {count} projects")
        
        results["EV-L02"] = {
            "status": "success",
            "result": f"{count} projects",
            "time": time.time() - start,
        }
    except Exception as e:
        print(f"❌ EV-L02 Failed: {e}")
        results["EV-L02"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_L02()

✅ EV-L02: Found 88 projects


In [ ]:
# EV-L03: In the GDC database, retrieve the primary sites represented across all projects
# Basic metadata retrieval (primary sites)
def eval_L03():
    start = time.time()
    try:
        result = rest_query("projects", {
            "size": "2000",
            "fields": "primary_site"
        })
        
        primary_sites = set()
        for project in result["data"]["hits"]:
            sites = project.get("primary_site", [])
            if isinstance(sites, list):
                for site in sites:
                    if site and site != "_missing":  # Exclude _missing values
                        primary_sites.add(site)
            elif sites and sites != "_missing":  # Exclude _missing values
                primary_sites.add(sites)
        
        count = len(primary_sites)
        sorted_sites = sorted(primary_sites)
        
        print(f"✅ EV-L03: Found {count} primary sites (excluding '_missing')")
        print(f"Sites: {'; '.join(sorted_sites)}")
        
        results["EV-L03"] = {
            "status": "success",
            "result": f"{count} primary sites",
            "data": sorted_sites,
            "time": time.time() - start
        }
    except Exception as e:
        print(f"❌ EV-L03 Failed: {e}")
        results["EV-L03"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_L03()

✅ EV-L03: Found 69 primary sites (excluding '_missing')
Sites: Accessory sinuses; Adrenal gland; Anus and anal canal; Base of tongue; Bladder; Bones, joints and articular cartilage of limbs; Bones, joints and articular cartilage of other and unspecified sites; Brain; Breast; Bronchus and lung; Cervix uteri; Colon; Connective, subcutaneous and other soft tissues; Corpus uteri; Esophagus; Eye and adnexa; Floor of mouth; Gallbladder; Gum; Heart, mediastinum, and pleura; Hematopoietic and reticuloendothelial systems; Hypopharynx; Kidney; Larynx; Lip; Liver and intrahepatic bile ducts; Lymph nodes; Meninges; Nasal cavity and middle ear; Nasopharynx; Not Reported; Oropharynx; Other and ill-defined digestive organs; Other and ill-defined sites; Other and ill-defined sites in lip, oral cavity and pharynx; Other and ill-defined sites within respiratory system and intrathoracic organs; Other and unspecified female genital organs; Other and unspecified major salivary glands; Other and unspecified

In [ ]:
# EV-L04: In the GDC database, list all data categories (e.g., Raw Sequencing Data, Transcriptome Profiling)
# Single-field lookup (data categories)
def eval_L04():
    start = time.time()
    try:
        result = rest_query("files", {
            "size": "0",
            "facets": "data_category"
        })
        
        categories = []
        for bucket in result["data"]["aggregations"]["data_category"]["buckets"]:
            categories.append(bucket["key"])
        
        count = len(categories)
        print(f"✅ EV-L04: Found {count} data categories")
        print(f"Categories: {'; '.join(sorted(categories))}")
        
        results["EV-L04"] = {
            "status": "success", 
            "result": f"{count} data categories",
            "data": sorted(categories),
            "time": time.time() - start
        }
    except Exception as e:
        print(f"❌ EV-L04 Failed: {e}")
        results["EV-L04"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_L04()

✅ EV-L04: Found 11 data categories
Categories: biospecimen; clinical; combined nucleotide variation; copy number variation; dna methylation; proteome profiling; sequencing reads; simple nucleotide variation; somatic structural variation; structural variation; transcriptome profiling


In [ ]:
# EV-L05: In the GDC database, get all experimental strategies used
# Direct enumeration (experimental strategies)
def eval_L05():
    start = time.time()
    try:
        result = rest_query("files", {
            "size": "0",
            "facets": "experimental_strategy"
        })
        
        strategies = []
        for bucket in result["data"]["aggregations"]["experimental_strategy"]["buckets"]:
            strategy = bucket["key"]
            if strategy and strategy != "_missing":  # Exclude _missing values
                strategies.append(strategy)
        
        count = len(strategies)
        print(f"✅ EV-L05: Found {count} experimental strategies (excluding '_missing')")
        print(f"Strategies: {'; '.join(sorted(strategies))}")
        
        results["EV-L05"] = {
            "status": "success",
            "result": f"{count} experimental strategies", 
            "data": sorted(strategies),
            "time": time.time() - start
        }
    except Exception as e:
        print(f"❌ EV-L05 Failed: {e}")
        results["EV-L05"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_L05()

✅ EV-L05: Found 13 experimental strategies (excluding '_missing')
Strategies: ATAC-Seq; Diagnostic Slide; Expression Array; Genotyping Array; Methylation Array; RNA-Seq; Reverse Phase Protein Array; Targeted Sequencing; Tissue Slide; WGS; WXS; miRNA-Seq; scRNA-Seq


In [ ]:
# EV-L06: In the GDC database, count the total number of RNA-Seq files across all projects
# Single filter condition (experimental_strategy = RNA-Seq)
def eval_L06():
    start = time.time()
    try:
        filters = {
            "op": "=",
            "content": {
                "field": "experimental_strategy",
                "value": "RNA-Seq"
            }
        }
        
        result = rest_query("files", {
            "filters": json.dumps(filters),
            "size": "0"
        })
        
        count = result["data"]["pagination"]["total"]
        print(f"✅ EV-L06: Found {count:,} RNA-Seq files across all projects")
        
        results["EV-L06"] = {
            "status": "success",
            "result": f"{count} RNA-Seq files",
            "time": time.time() - start
        }
    except Exception as e:
        print(f"❌ EV-L06 Failed: {e}")
        results["EV-L06"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_L06()

✅ EV-L06: Found 233,388 RNA-Seq files across all projects


In [ ]:
# EV-L07: In the GDC database, list all annotation categories
# Simple lookup (annotation categories)
def eval_L07():
    start = time.time()
    try:
        result = rest_query("annotations", {
            "size": "0",
            "facets": "category"
        })
        
        categories = []
        
        for bucket in result["data"]["aggregations"]["category"]["buckets"]:
            categories.append(bucket["key"])
        
        print(f"✅ EV-L07: Found {len(categories)} annotation categories")
        print(f"Categories: {'; '.join(sorted(categories))}")
        
        results["EV-L07"] = {
            "status": "success",
            "result": f"{len(categories)} annotation categories",
            "data": {"categories": sorted(categories)},
            "time": time.time() - start
        }
    except Exception as e:
        print(f"❌ EV-L07 Failed: {e}")
        results["EV-L07"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_L07()

✅ EV-L07: Found 35 annotation categories
Categories: acceptable treatment for tcga tumor; alternate sample pipeline; barcode incorrect; bcr notification; biospecimen identity unknown; case submitted is found to be a recurrence after submission; center qc failed; duplicate case; duplicate item; general; genotype mismatch; history of acceptable prior treatment related to a prior/other malignancy; history of unacceptable prior treatment related to a prior/other malignancy; inadvertently shipped; item does not meet study protocol; item flagged dnu; item flagged low quality; item in special subset; item is noncanonical; item may not meet study protocol; molecular analysis outside specification; neoadjuvant therapy; normal class but appears diseased; normal tissue origin incorrect; pathology outside specification; permanently missing item or object; prior malignancy; qualification metrics changed; qualified in error; sample compromised; subject identity unknown; subject withdrew consent; syn

In [ ]:
# EV-L08: In the GDC database, list all the available disease types
def eval_L08():
    start = time.time()
    try:
        result = rest_query("cases", {
            "size": "0",
            "facets": "disease_type"
        })
        
        disease_types = []
        for bucket in result["data"]["aggregations"]["disease_type"]["buckets"]:
            disease_type = bucket["key"]
            if disease_type and disease_type != "_missing":  # Exclude _missing values
                disease_types.append(disease_type)
        
        count = len(disease_types)
        print(f"✅ EV-L08: Found {count} disease types")
        print(f"Disease types: {'; '.join(sorted(disease_types))}")
        
        results["EV-L08"] = {
            "status": "success",
            "result": f"{count} disease types",
            "data": sorted(disease_types),
            "time": time.time() - start,
        }
    except Exception as e:
        print(f"❌ EV-L08 Failed: {e}")
        results["EV-L08"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_L08()

✅ EV-L08: Found 48 disease types
Disease types: acinar cell neoplasms; acute lymphoblastic leukemia; adenomas and adenocarcinomas; adnexal and skin appendage neoplasms; basal cell neoplasms; blood vessel tumors; chronic myeloproliferative disorders; complex epithelial neoplasms; complex mixed and stromal neoplasms; cystic, mucinous and serous neoplasms; ductal and lobular neoplasms; epithelial neoplasms, nos; fibroepithelial neoplasms; fibromatous neoplasms; germ cell neoplasms; gliomas; granular cell tumors and alveolar soft part sarcomas; leukemias, nos; lipomatous neoplasms; lymphoid leukemias; malignant lymphomas, nos or diffuse; mature b-cell lymphomas; mature t- and nk-cell lymphomas; meningiomas; mesothelial neoplasms; miscellaneous bone tumors; miscellaneous tumors; mucoepidermoid neoplasms; myelodysplastic syndromes; myeloid leukemias; myomatous neoplasms; neoplasms, nos; nerve sheath tumors; neuroepitheliomatous neoplasms; nevi and melanomas; not applicable; not reported; odo

In [ ]:
# EV-L09: In the GDC database, list the available ethnicity categories
# Single-field enumeration (ethnicity categories)
def eval_L09():
    start = time.time()
    try:
        result = rest_query("cases", {
            "size": "0",
            "facets": "demographic.ethnicity"
        })
        
        ethnicities = []
        for bucket in result["data"]["aggregations"]["demographic.ethnicity"]["buckets"]:
            ethnicity = bucket["key"]
            if ethnicity and ethnicity != "_missing":  # Exclude _missing values
                ethnicities.append(ethnicity)
        
        count = len(ethnicities)
        print(f"✅ EV-L09: Found {count} ethnicity categories")
        print(f"Ethnicities: {'; '.join(sorted(ethnicities))}")
        
        results["EV-L09"] = {
            "status": "success",
            "result": f"{count} ethnicity categories",
            "data": sorted(ethnicities),
            "time": time.time() - start
        }
    except Exception as e:
        print(f"❌ EV-L09 Failed: {e}")
        results["EV-L09"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_L09()

✅ EV-L09: Found 4 ethnicity categories
Ethnicities: hispanic or latino; not hispanic or latino; not reported; unknown


In [ ]:
# EV-L10: In the GDC database, what are the available platform types used for sequencing
# Basic metadata with counts (platform types)
def eval_L10():
    start = time.time()
    try:
        result = rest_query("files", {
            "size": "0",
            "facets": "platform"
        })
        
        platforms = []
        platform_counts = {}
        for bucket in result["data"]["aggregations"]["platform"]["buckets"]:
            platform = bucket["key"]
            count = bucket["doc_count"]
            if platform and platform != "_missing":  # Exclude _missing values
                platforms.append(platform)
                platform_counts[platform] = count
        
        total_platforms = len(platforms)
        print(f"✅ EV-L10: Found {total_platforms} platform types (excluding '_missing')")
        
        # Show top platforms by count
        sorted_platforms = sorted(platform_counts.items(), key=lambda x: x[1], reverse=True)
        for platform, count in sorted_platforms[:10]:
            print(f"  {platform}: {count:,}")
        
        results["EV-L10"] = {
            "status": "success",
            "result": f"{total_platforms} platform types",
            "data": {"platforms": platforms, "counts": platform_counts},
            "time": time.time() - start,
        }
    except Exception as e:
        print(f"❌ EV-L10 Failed: {e}")
        results["EV-L10"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_L10()

✅ EV-L10: Found 10 platform types (excluding '_missing')
  illumina: 799,977
  affymetrix snp 6.0: 147,734
  illumina human methylation 450: 31,776
  illumina methylation epic: 18,522
  illumina human methylation 27: 9,435
  rppa: 7,906
  genechip u133a: 1,243
  illumina methylation epic v2: 1,179
  complete genomics: 581
  genechip u133 plus 2.0: 183


### ENTITY FILTERING QUERIES (Medium Complexity)

These queries apply specific filtering criteria to narrow down results within one or two entity types.

In [ ]:
# EV-M01: In the GDC database, count male vs. female cases in TCGA-LUAD
# Cross-entity filter with faceting (project + gender)
def eval_M01():
    start = time.time()
    try:
        filters = {
            "op": "=",
            "content": {
                "field": "project.project_id",
                "value": "TCGA-LUAD"
            }
        }
        
        result = rest_query("cases", {
            "filters": json.dumps(filters),
            "size": "0",
            "facets": "demographic.gender"
        })
        
        gender_counts = {}
        for bucket in result["data"]["aggregations"]["demographic.gender"]["buckets"]:
            gender_counts[bucket["key"]] = bucket["doc_count"]
        
        females = gender_counts.get("female", 0)
        males = gender_counts.get("male", 0)
        
        print(f"✅ EV-M01: TCGA-LUAD gender distribution:")
        print(f"  Females: {females}")
        print(f"  Males: {males}")
        
        results["EV-M01"] = {
            "status": "success",
            "result": f"{females} females, {males} males",
            "data": gender_counts,
            "time": time.time() - start,
        }
    except Exception as e:
        print(f"❌ EV-M01 Failed: {e}")
        results["EV-M01"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_M01()

✅ EV-M01: TCGA-LUAD gender distribution:
  Females: 280
  Males: 242


In [ ]:
# EV-M02: In the GDC database, show the race distribution for TCGA-LIHC
# Project-specific demographic distribution
def eval_M02():
    start = time.time()
    try:
        filters = {
            "op": "=",
            "content": {
                "field": "project.project_id",
                "value": "TCGA-LIHC"
            }
        }
        
        result = rest_query("cases", {
            "filters": json.dumps(filters),
            "size": "0",
            "facets": "demographic.race"
        })
        
        # Get race distribution
        race_counts = {}
        total_cases = 0
        for bucket in result["data"]["aggregations"]["demographic.race"]["buckets"]:
            race = bucket["key"]
            count = bucket["doc_count"]
            race_counts[race] = count
            total_cases += count
        
        print(f"✅ EV-M02: Race distribution for TCGA-LIHC ({total_cases} total cases):")
        for race, count in sorted(race_counts.items(), key=lambda x: x[1], reverse=True):
            percentage = (count / total_cases) * 100 if total_cases > 0 else 0
            print(f"  {race}: {count} ({percentage:.2f}%)")
        
        results["EV-M02"] = {
            "status": "success",
            "result": f"Race distribution for TCGA-LIHC",
            "data": race_counts,
            "time": time.time() - start
        }
    except Exception as e:
        print(f"❌ EV-M02 Failed: {e}")
        results["EV-M02"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_M02()

✅ EV-M02: Race distribution for TCGA-LIHC (377 total cases):
  white: 187 (49.60%)
  asian: 161 (42.71%)
  black or african american: 17 (4.51%)
  not reported: 6 (1.59%)
  unknown: 4 (1.06%)
  american indian or alaska native: 2 (0.53%)


In [ ]:
# EV-M03: In the GDC database, count cases diagnosed at any Stage III variant (Stage III, Stage IIIA, Stage IIIB) in the TCGA-LUAD project
# OR logic across multiple stage values with individual counts
def eval_M03():
    start = time.time()
    try:
        stage_variants = ["Stage III", "Stage IIIA", "Stage IIIB"]
        filters = {
            "op": "and",
            "content": [
                {"op": "=", "content": {"field": "project.project_id", "value": "TCGA-LUAD"}},
                {
                    "op": "in",
                    "content": {
                        "field": "diagnoses.ajcc_pathologic_stage",
                        "value": stage_variants
                    }
                }
            ]
        }
        
        # Get total count and breakdown by stage
        result = rest_query("cases", {
            "filters": json.dumps(filters),
            "size": "0",
            "facets": "diagnoses.ajcc_pathologic_stage"
        })
        
        total_count = result["data"]["pagination"]["total"]
        
        # Get individual counts for each stage variant
        stage_counts = {stage.lower().replace(" ", " "): 0 for stage in stage_variants}
        for bucket in result["data"]["aggregations"]["diagnoses.ajcc_pathologic_stage"]["buckets"]:
            stage = bucket["key"]
            count = bucket["doc_count"]
            # Match against our target stages (case-insensitive)
            if "iiia" in stage.lower():
                stage_counts["Stage IIIA"] = count
            elif "iiib" in stage.lower():
                stage_counts["Stage IIIB"] = count
            elif stage.lower() == "stage iii":
                stage_counts["Stage III"] = count
        
        print(f"✅ EV-M03: Found {total_count} TCGA-LUAD cases with Stage III variants")
        print(f"  Individual counts:")
        for stage in stage_variants:
            count = stage_counts.get(stage, 0)
            print(f"    {stage}: {count}")
        
        results["EV-M03"] = {
            "status": "success",
            "result": f"{total_count} TCGA-LUAD Stage III cases",
            "data": {"total": total_count, "stage_counts": stage_counts},
            "time": time.time() - start
        }
    except Exception as e:
        print(f"❌ EV-M03 Failed: {e}")
        results["EV-M03"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_M03()

✅ EV-M03: Found 86 TCGA-LUAD cases with Stage III variants
  Individual counts:
    Stage III: 1
    Stage IIIA: 74
    Stage IIIB: 11


In [ ]:
# EV-M04: In the GDC database, count cases with tumor stage T3 or T4 in the TCGA-LUAD project
# OR logic on tumor stage field
def eval_M04():
    start = time.time()
    try:
        filters = {
            "op": "and",
            "content": [
                {"op": "=", "content": {"field": "project.project_id", "value": "TCGA-LUAD"}},
                {
                    "op": "in",
                    "content": {
                        "field": "diagnoses.ajcc_pathologic_t",
                        "value": ["T3", "T4"]
                    }
                }
            ]
        }
        
        result = rest_query("cases", {
            "filters": json.dumps(filters),
            "size": "0",
            "facets": "diagnoses.ajcc_pathologic_t"
        })
        
        count = result["data"]["pagination"]["total"]
        
        # Get individual counts for T3 and T4
        stage_counts = {}
        for bucket in result["data"]["aggregations"]["diagnoses.ajcc_pathologic_t"]["buckets"]:
            stage = bucket["key"]
            if stage.lower() in ["t3", "t4"]:
                stage_counts[stage.upper()] = bucket["doc_count"]
        
        print(f"✅ EV-M04: Found {count} TCGA-LUAD cases with tumor stage T3 or T4")
        print(f"  Individual counts:")
        print(f"    T3: {stage_counts.get('T3', 0)}")
        print(f"    T4: {stage_counts.get('T4', 0)}")
        
        results["EV-M04"] = {
            "status": "success",
            "result": f"{count} TCGA-LUAD T3/T4 cases",
            "data": {"total": count, "stage_counts": stage_counts},
            "time": time.time() - start,
        }
    except Exception as e:
        print(f"❌ EV-M04 Failed: {e}")
        results["EV-M04"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_M04()

✅ EV-M04: Found 69 TCGA-LUAD cases with tumor stage T3 or T4
  Individual counts:
    T3: 50
    T4: 19


In [ ]:
# EV-M05: In the GDC database, count WXS files that are larger than 10 GB (10,737,418,240 bytes) in the CPTAC-3 project
# Multiple filters (project + experimental_strategy + file_size)
def eval_M05():
    start = time.time()
    try:
        filters = {
            "op": "and",
            "content": [
                {"op": "=", "content": {"field": "cases.project.project_id", "value": "CPTAC-3"}},
                {"op": "=", "content": {"field": "experimental_strategy", "value": "WXS"}},
                {"op": ">", "content": {"field": "file_size", "value": 10737418240}}  # 10 GB in bytes
            ]
        }
        
        result = rest_query("files", {
            "filters": json.dumps(filters),
            "size": "0"
        })
        
        count = result["data"]["pagination"]["total"]
        print(f"✅ EV-M05: Found {count} WXS files > 10GB in CPTAC-3")
        
        results["EV-M05"] = {
            "status": "success",
            "result": f"{count} WXS files > 10GB",
            "time": time.time() - start
        }
    except Exception as e:
        print(f"❌ EV-M05 Failed: {e}")
        results["EV-M05"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_M05()

✅ EV-M05: Found 4256 WXS files > 10GB in CPTAC-3


In [ ]:
# EV-M06: In the GDC database, count cases with primary site 'Bronchus and lung' AND age at diagnosis greater than 70 years (25,550 days)
# Two simultaneous filters (site + age calculation)
def eval_M06():
    start = time.time()
    try:
        filters = {
            "op": "and",
            "content": [
                {"op": "in", "content": {"field": "primary_site", "value": ["Bronchus and lung"]}},
                {"op": ">", "content": {"field": "diagnoses.age_at_diagnosis", "value": 25550}}  # 70 years in days
            ]
        }
        
        result = rest_query("cases", {
            "filters": json.dumps(filters),
            "size": "0"
        })
        
        count = result["data"]["pagination"]["total"]
        print(f"✅ EV-M06: Found {count} cases with primary site 'Bronchus and lung' AND age > 70 years")
        
        results["EV-M06"] = {
            "status": "success",
            "result": f"{count} lung cases with age > 70",
            "time": time.time() - start
        }
    except Exception as e:
        print(f"❌ EV-M06 Failed: {e}")
        results["EV-M06"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_M06()

✅ EV-M06: Found 1632 cases with primary site 'Bronchus and lung' AND age > 70 years


In [ ]:
# EV-M07: In the GDC database, count the number of cases from the TCGA-OV project where the patient died within less than 1000 days
# Range-based filtering (project + days_to_death < 1000)
def eval_M07():
    start = time.time()
    try:
        filters = {
            "op": "and",
            "content": [
                {"op": "=", "content": {"field": "project.project_id", "value": "TCGA-OV"}},
                {"op": "<", "content": {"field": "demographic.days_to_death", "value": 1000}}
            ]
        }
        
        result = rest_query("cases", {
            "filters": json.dumps(filters),
            "size": "0"
        })
        
        count = result["data"]["pagination"]["total"]
        print(f"✅ EV-M07: Found {count} TCGA-OV cases with days_to_death < 1000")
        
        results["EV-M07"] = {
            "status": "success",
            "result": f"{count} TCGA-OV cases with days_to_death < 1000",
            "time": time.time() - start
        }
    except Exception as e:
        print(f"❌ EV-M07 Failed: {e}")
        results["EV-M07"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_M07()

✅ EV-M07: Found 154 TCGA-OV cases with days_to_death < 1000


In [ ]:
# EV-M08: In the GDC database, retrieve case IDs and their associated file IDs for patients that meet BOTH criteria: (1) belong to the TCGA-LUAD project, AND (2) have a diagnosis with AJCC pathologic stage equal to Stage III only
# Complex relationship mapping (stage-specific cases with file associations) 
def eval_M08():
    start = time.time()
    try:
        filters = {
            "op": "and",
            "content": [
                {"op": "=", "content": {"field": "project.project_id", "value": "TCGA-LUAD"}},
                {"op": "=", "content": {"field": "diagnoses.ajcc_pathologic_stage", "value": "Stage III"}}
            ]
        }
        
        # Get cases with Stage III LUAD
        cases_result = rest_query("cases", {
            "filters": json.dumps(filters),
            "size": "1000",
            "fields": "submitter_id,case_id,files.file_id"
        })
        
        stage_iii_cases = []
        all_file_ids = []
        
        for case in cases_result["data"]["hits"]:
            case_files = []
            files = case.get("files", [])
            
            for file_info in files:
                file_id = file_info.get("file_id")
                if file_id:
                    case_files.append(file_id)
                    all_file_ids.append(file_id)
            
            case_info = {
                "case_id": case["case_id"],
                "submitter_id": case["submitter_id"],
                "file_ids": case_files,
                "file_count": len(case_files)
            }
            stage_iii_cases.append(case_info)
        
        cases_count = len(stage_iii_cases)
        total_files = len(all_file_ids)
        
        print(f"✅ EV-M08: Found {cases_count} LUAD Stage III cases with {total_files} files")
        
        if stage_iii_cases:
            print(f"  Sample cases:")
            for case in stage_iii_cases[:3]:
                print(f"    Case {case['submitter_id']}: {case['file_count']} files")
        
        results["EV-M08"] = {
            "status": "success",
            "result": f"{cases_count} LUAD Stage III cases with {total_files} files",
            "data": {"cases": stage_iii_cases, "total_files": total_files},
            "time": time.time() - start
        }
    except Exception as e:
        print(f"❌ EV-M08 Failed: {e}")
        results["EV-M08"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_M08()

✅ EV-M08: Found 1 LUAD Stage III cases with 77 files
  Sample cases:
    Case TCGA-95-7947: 77 files


In [ ]:
# EV-M09: In the GDC database, list all projects that have more than 500 cases
# Aggregation with threshold filtering
def eval_M09():
    start = time.time()
    try:
        result = rest_query("cases", {
            "size": "0",
            "facets": "project.project_id"
        })
        
        # Get project counts and filter those > 500
        large_projects = []
        for bucket in result["data"]["aggregations"]["project.project_id"]["buckets"]:
            project = bucket["key"]
            count = bucket["doc_count"]
            if count > 500:
                large_projects.append((project, count))
        
        # Sort by count descending
        large_projects.sort(key=lambda x: x[1], reverse=True)
        
        print(f"✅ EV-M09: Found {len(large_projects)} projects with more than 500 cases:")
        for project, count in large_projects:
            print(f"  {project}: {count:,} cases")
        
        results["EV-M09"] = {
            "status": "success",
            "result": f"{len(large_projects)} projects with > 500 cases",
            "data": large_projects,
            "time": time.time() - start
        }
    except Exception as e:
        print(f"❌ EV-M09 Failed: {e}")
        results["EV-M09"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_M09()

✅ EV-M09: Found 21 projects with more than 500 cases:
  FM-AD: 18,004 cases
  CCDI-MCI: 3,093 cases
  TARGET-AML: 2,492 cases
  CPTAC-3: 1,683 cases
  TARGET-ALL-P2: 1,587 cases
  MP2PRT-ALL: 1,510 cases
  TARGET-NBL: 1,132 cases
  TCGA-BRCA: 1,098 cases
  MMRF-COMMPASS: 995 cases
  BEATAML1.0-COHORT: 826 cases
  HCMI-CMDC: 804 cases
  TARGET-WT: 652 cases
  TCGA-GBM: 617 cases
  TCGA-OV: 608 cases
  TCGA-LUAD: 585 cases
  TCGA-UCEC: 560 cases
  TCGA-KIRC: 537 cases
  TCGA-HNSC: 528 cases
  TCGA-LGG: 516 cases
  TCGA-THCA: 507 cases
  TCGA-LUSC: 504 cases


In [ ]:
# EV-M10: In the GDC database, count valid smoking duration records and the distribution of years of smoking for TCGA-LUSC project cases
# Domain-specific analysis (smoking duration distribution)
def eval_M10():
    start = time.time()
    try:
        # Use GraphQL to get exposure data
        query = """
        query LUSCSmokingData($filters: FiltersArgument) {
          viewer {
            repository {
              cases {
                hits(first: 600, filters: $filters) {
                  edges {
                    node {
                      case_id
                      exposures {
                        hits {
                          edges {
                            node {
                              tobacco_smoking_onset_year
                              tobacco_smoking_quit_year
                            }
                          }
                        }
                      }
                    }
                  }
                }
              }
            }
          }
        }
        """
        
        variables = {
            "filters": {
                "op": "=",
                "content": {
                    "field": "project.project_id",
                    "value": "TCGA-LUSC"
                }
            }
        }
        
        result = graphql_query(query, variables)
        if result:
            # Process exposure data
            smoking_years = []
            cases_processed = 0
            
            for case_edge in result["data"]["viewer"]["repository"]["cases"]["hits"]["edges"]:
                case_node = case_edge.get("node", {})
                exposures = case_node.get("exposures", {}).get("hits", {}).get("edges", [])
                
                for exp_edge in exposures:
                    exp = exp_edge.get("node", {})
                    onset = exp.get("tobacco_smoking_onset_year")
                    quit = exp.get("tobacco_smoking_quit_year")
                    
                    if onset and quit:
                        years = quit - onset
                        if years > 0:
                            smoking_years.append(years)
                
                cases_processed += 1
            
            print(f"✅ EV-M10: Processed {cases_processed} TCGA-LUSC cases")
            print(f"  Found {len(smoking_years)} valid smoking duration records")
            
            if smoking_years:
                # Create distribution bins
                bins = [0, 10, 20, 30, 40, 50, 100]
                bin_counts = [0] * (len(bins) - 1)
                
                for years in smoking_years:
                    for i in range(len(bins) - 1):
                        if bins[i] <= years < bins[i+1]:
                            bin_counts[i] += 1
                            break
                
                print(f"  Smoking years distribution:")
                for i in range(len(bins)-1):
                    print(f"    {bins[i]}-{bins[i+1]} years: {bin_counts[i]} cases")
            
            results["EV-M10"] = {
                "status": "success",
                "result": f"Smoking duration distribution for TCGA-LUSC",
                "data": {"smoking_years": smoking_years, "cases_processed": cases_processed},
                "time": time.time() - start
            }
        else:
            results["EV-M10"] = {"status": "error", "error": "GraphQL query failed", "time": time.time() - start}
            
    except Exception as e:
        print(f"❌ EV-M10 Failed: {e}")
        results["EV-M10"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_M10()

✅ EV-M10: Processed 504 TCGA-LUSC cases
  Found 224 valid smoking duration records
  Smoking years distribution:
    0-10 years: 2 cases
    10-20 years: 7 cases
    20-30 years: 34 cases
    30-40 years: 46 cases
    40-50 years: 69 cases
    50-100 years: 66 cases


### COMPLEX COHORTS QUERIES (High Complexity)

These queries require multi-step reasoning, multiple entity relationships, or sophisticated filtering to define patient/sample cohorts.

In [ ]:
# EV-H01: In the GDC database, count files that meet ALL three criteria: (1) TCGA-GBM project, (2) WGS experimental strategy, AND (3) file size > 50 GB
# Multiple filters with range (project + strategy + file_size)
def eval_H01():
    start = time.time()
    try:
        filters = {
            "op": "and",
            "content": [
                {"op": "=", "content": {"field": "cases.project.project_id", "value": "TCGA-GBM"}},
                {"op": "=", "content": {"field": "experimental_strategy", "value": "WGS"}},
                {"op": ">", "content": {"field": "file_size", "value": 53687091200}}  # 50 GB in bytes
            ]
        }

        result = rest_query("files", {
            "filters": json.dumps(filters),
            "size": "0"
        })
        
        count = result["data"]["pagination"]["total"]
        print(f"✅ EV-H01: Found {count} WGS files > 50GB for TCGA-GBM")
        
        results["EV-H01"] = {
            "status": "success",
            "result": f"{count} WGS files > 50GB",
            "time": time.time() - start
        }
    except Exception as e:
        print(f"❌ EV-H01 Failed: {e}")
        results["EV-H01"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_H01()

✅ EV-H01: Found 798 WGS files > 50GB for TCGA-GBM


In [ ]:
# EV-H02: In the GDC database, count cases that meet ALL criteria: breast cancer, female, age < 40, RNA-Seq files
# Multi-dimensional cohort (breast + female + age<40 + RNA-Seq)
def eval_H02():
    start = time.time()
    try:
        filters = {
            "op": "and",
            "content": [
                {"op": "in", "content": {"field": "primary_site", "value": ["Breast"]}},
                {"op": "=", "content": {"field": "demographic.gender", "value": "female"}},
                {"op": "<", "content": {"field": "diagnoses.age_at_diagnosis", "value": 14600}},  # 40 years in days
                {"op": "=", "content": {"field": "files.experimental_strategy", "value": "RNA-Seq"}}
            ]
        }
        
        result = rest_query("cases", {
            "filters": json.dumps(filters),
            "size": "0"
        })
        
        count = result["data"]["pagination"]["total"]
        print(f"✅ EV-H02: Found {count} breast cancer female cases under 40 with RNA-Seq")
        
        results["EV-H02"] = {
            "status": "success",
            "result": f"{count} breast cancer female cases under 40 with RNA-Seq",
            "time": time.time() - start
        }
    except Exception as e:
        print(f"❌ EV-H02 Failed: {e}")
        results["EV-H02"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_H02()

✅ EV-H02: Found 144 breast cancer female cases under 40 with RNA-Seq


In [ ]:
# EV-H03: In the GDC database, find cases with alcohol history AND AJCC Stage II
# Complex cohort definition combining exposure history and clinical staging
def eval_H03():
    start = time.time()
    try:
        filters = {
            "op": "and",
            "content": [
                {"op": "=", "content": {"field": "exposures.alcohol_history", "value": "Yes"}},
                {"op": "=", "content": {"field": "diagnoses.ajcc_pathologic_stage", "value": "Stage II"}}
            ]
        }
        
        # First, get the total count
        count_result = rest_query("cases", {
            "filters": json.dumps(filters),
            "size": "0"
        })
        total_count = count_result["data"]["pagination"]["total"]
        
        # Fetch all cases in batches
        all_cases = []
        size = 1000
        from_idx = 0
        
        while from_idx < total_count:
            result = rest_query("cases", {
                "filters": json.dumps(filters),
                "size": str(size),
                "from": str(from_idx),
                "fields": "submitter_id,case_id,project.project_id"
            })
            
            batch_cases = result["data"]["hits"]
            if not batch_cases:
                break
            
            for case in batch_cases:
                all_cases.append({
                    "case_id": case["case_id"],
                    "submitter_id": case["submitter_id"],
                    "project": case.get("project", {}).get("project_id", "Unknown")
                })
            
            from_idx += len(batch_cases)
            if len(batch_cases) < size:
                break
        
        count = len(all_cases)
        print(f"✅ EV-H03: Found {count} cases with alcohol history AND AJCC Stage II")
        
        # Group by project
        by_project = {}
        for case in all_cases:
            project = case["project"]
            if project not in by_project:
                by_project[project] = []
            by_project[project].append(case)
        
        # Display all cases grouped by project
        print(f"\nAll Case IDs:")
        print("="*80)
        for project in sorted(by_project.keys()):
            cases_in_project = by_project[project]
            print(f"\n{project} ({len(cases_in_project)} cases):")
            for case in cases_in_project:
                print(f"  {case['submitter_id']} ({case['case_id']})")
        print("="*80)
        
        results["EV-H03"] = {
            "status": "success",
            "result": f"{count} cases with alcohol history AND Stage II",
            "data": {"cases": all_cases},
            "time": time.time() - start
        }
    except Exception as e:
        print(f"❌ EV-H03 Failed: {e}")
        results["EV-H03"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_H03()

✅ EV-H03: Found 56 cases with alcohol history AND AJCC Stage II

All Case IDs:

CPTAC-3 (31 cases):
  C3N-03619 (94badce5-890f-468a-a689-7596a1acbd7d)
  C3L-02669 (a162aaa1-45e3-4294-90c2-1e2cd8a8b356)
  C3L-00981 (d84bfa68-63b1-4c84-848d-8b473638f970)
  C3L-04354 (db9021f9-df04-48c4-b56c-499f49c5212f)
  C3N-02296 (116436ce-6250-43b1-8fd2-2ebdb3ea9186)
  C3L-02651 (3ef779a0-457a-4045-b04c-191b734b81a4)
  C3L-04759 (3f92c203-a319-46a6-a627-8027070961fc)
  C3N-04273 (701129b7-d95a-4fac-a9be-0d0da0642a81)
  C3L-02802 (2a971d49-1fcc-40eb-a02b-0736a0929f43)
  C3N-01648 (3a420b74-ac84-470e-99f1-811cbc37ac5e)
  C3N-04280 (1f370a2d-7fbd-4fa5-8bb8-bdaa94c1ba9f)
  C3N-03457 (1f9bd893-4a8c-47ae-aa59-c32b5a430575)
  C3N-02789 (4d6794c9-25bc-47a0-ac2e-352d434b577c)
  C3L-00994 (5119c0a8-d0ef-4ec2-b58c-0ff9e0c8f1db)
  C3N-03458 (28d777f2-f4c8-4853-bd28-1bba94f2bfa6)
  C3N-04275 (2dc5fc3a-4f92-4b1b-8a54-bd86bbdeac93)
  C3L-00991 (61665776-139e-450f-bc64-eaf29bd7ff25)
  C3N-00519 (eae77c25-2317-4ec4-8

In [ ]:
# EV-H04: In the GDC database, count files for cases with cancer-related death AND died ≤50 years
# Multiple criteria with file counting
def eval_H04():
    start = time.time()
    try:
        filters = {
            "op": "and",
            "content": [
                {"op": "=", "content": {"field": "demographic.cause_of_death", "value": "Cancer Related"}},
                {"op": "<=", "content": {"field": "demographic.days_to_death", "value": 18250}}  # 50 years in days
            ]
        }
        
        # Get cases
        cases_result = rest_query("cases", {
            "filters": json.dumps(filters),
            "size": "0"
        })
        cases_count = cases_result["data"]["pagination"]["total"]
        
        # Get files for these cases
        files_result = rest_query("files", {
            "filters": json.dumps({
                "op": "and",
                "content": [
                    {"op": "=", "content": {"field": "cases.demographic.cause_of_death", "value": "Cancer Related"}},
                    {"op": "<=", "content": {"field": "cases.demographic.days_to_death", "value": 18250}}
                ]
            }),
            "size": "0"
        })
        files_count = files_result["data"]["pagination"]["total"]
        
        print(f"✅ EV-H04: Found {files_count:,} files for {cases_count:,} cases")
        print(f"  Cases: Cancer-related deaths ≤50 years old")
        
        results["EV-H04"] = {
            "status": "success",
            "result": f"{files_count} files for {cases_count} cases",
            "data": {"cases_count": cases_count, "files_count": files_count},
            "time": time.time() - start
        }
    except Exception as e:
        print(f"❌ EV-H04 Failed: {e}")
        results["EV-H04"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_H04()

✅ EV-H04: Found 62,935 files for 1,188 cases
  Cases: Cancer-related deaths ≤50 years old


In [6]:
# EV-H05: In the GDC database, find and list all cases that meet BOTH of these criteria: (1) have a documented family history where a relative's primary diagnosis was "Breast Cancer" (family_histories.relationship_primary_diagnosis = "Breast Cancer"), AND (2) have at least one associated file that was generated using the RNA-Seq experimental strategy (files.experimental_strategy = "RNA-Seq"). Return the case identifiers, total count, and breakdown by project.
# Multi-entity join with family history and file type filtering
def eval_H05():
    start = time.time()
    try:
        filters = {
            "op": "and",
            "content": [
                {"op": "=", "content": {"field": "family_histories.relationship_primary_diagnosis", "value": "Breast Cancer"}},
                {"op": "=", "content": {"field": "files.experimental_strategy", "value": "RNA-Seq"}}
            ]
        }
        
        # Get total count first
        count_result = rest_query("cases", {
            "filters": json.dumps(filters),
            "size": "0"
        })
        total = count_result["data"]["pagination"]["total"]
        
        # Fetch all cases in batches
        all_cases = []
        size = 1000
        from_idx = 0
        
        while from_idx < total:
            result = rest_query("cases", {
                "filters": json.dumps(filters),
                "size": str(size),
                "from": str(from_idx),
                "fields": "case_id,submitter_id,project.project_id"
            })
            
            batch_cases = result["data"]["hits"]
            if not batch_cases:
                break
            
            all_cases.extend(batch_cases)
            from_idx += len(batch_cases)
            
            if len(batch_cases) < size:
                break
        
        # Group by project
        by_project = {}
        for case in all_cases:
            project = case.get('project', {}).get('project_id', 'Unknown')
            if project not in by_project:
                by_project[project] = []
            by_project[project].append(case)
        
        print(f"✅ EV-H05: Found {total:,} cases with family history of breast cancer AND RNA-Seq files")
        print(f"\n  Breakdown by project:")
        for project in sorted(by_project.keys()):
            print(f"    {project}: {len(by_project[project]):,} cases")
        
        print(f"\n  All Case IDs:")
        for project in sorted(by_project.keys()):
            print(f"\n  {project}:")
            for case in by_project[project]:
                print(f"    {case.get('submitter_id')} ({case.get('case_id')})")
        
        results["EV-H05"] = {
            "status": "success",
            "result": f"{total} cases with family history of breast cancer AND RNA-Seq",
            "data": {
                "total": total,
                "by_project": {project: len(cases) for project, cases in by_project.items()},
                "cases": [{"case_id": c.get('case_id'), "submitter_id": c.get('submitter_id'), 
                          "project": c.get('project', {}).get('project_id', 'Unknown')} 
                         for c in all_cases]
            },
            "time": time.time() - start
        }
    except Exception as e:
        print(f"❌ EV-H05 Failed: {e}")
        results["EV-H05"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_H05()

✅ EV-H05: Found 144 cases with family history of breast cancer AND RNA-Seq files

  Breakdown by project:
    HCMI-CMDC: 4 cases
    MMRF-COMMPASS: 71 cases
    TCGA-BLCA: 30 cases
    TCGA-CHOL: 6 cases
    TCGA-MESO: 7 cases
    TCGA-PAAD: 10 cases
    TCGA-TGCT: 15 cases
    TCGA-UVM: 1 cases

  All Case IDs:

  HCMI-CMDC:
    HCM-CSHL-0153-C50 (ad400773-3306-4e97-bda6-f03fcc471692)
    HCM-CSHL-0261-C50 (9323fd2e-57cb-4149-ba36-274921d70215)
    HCM-CSHL-0463-C50 (f2461357-8aa6-461b-9a13-1f18d44e50e9)
    HCM-CSHL-0514-C50 (d7b49708-3dd6-4837-856a-bce8ceb8e76f)

  MMRF-COMMPASS:
    MMRF_2559 (01a01acf-ed76-4f04-ad6a-2e66f29754dc)
    MMRF_1184 (02e02ba3-39e8-4c62-ae9f-25fe5c9d44c1)
    MMRF_2485 (0afc2090-c198-4ca6-b0c4-cb93b10b01f6)
    MMRF_2419 (175138f4-b09b-4528-96fb-2a22cfcecd2b)
    MMRF_2002 (18724e8a-1746-4420-8eba-eb716222722c)
    MMRF_2054 (193de550-e410-4dfb-ae82-0f67f8644d97)
    MMRF_1985 (1d503561-6913-4a0e-a1f1-615c6dc315a1)
    MMRF_2501 (21a0cabb-c588-4a2c-bd54-

In [6]:
# EV-H06: In the GDC database, calculate and report the male and female mean survival times in years for TCGA-LIHC cases.
def eval_H06():
    start = time.time()
    try:
        print("Analyzing gender survival differences in TCGA-LIHC...")
        
        # Get TCGA-LIHC cases with survival and gender data
        lihc_filter = {
            "op": "and",
            "content": [
                {"op": "=", "content": {"field": "project.project_id", "value": "TCGA-LIHC"}},
                {"op": "not", "content": {"field": "demographic.days_to_death", "value": "_missing"}},
                {"op": "not", "content": {"field": "demographic.gender", "value": "_missing"}}
            ]
        }
        
        # Get detailed case data
        cases = rest_query("cases", {
            "filters": json.dumps(lihc_filter),
            "fields": "demographic.gender,demographic.days_to_death,demographic.vital_status,diagnoses.age_at_diagnosis",
            "size": "2000"
        })
        
        # Extract survival data by gender for all cases with days_to_death data
        male_survival = []
        female_survival = []
        
        for case in cases["data"]["hits"]:
            demo = case.get("demographic", {})
            
            days = demo.get("days_to_death")
            gender = demo.get("gender")
            
            if days and gender in ["male", "female"]:
                if gender == "male":
                    male_survival.append(days)
                else:
                    female_survival.append(days)
        
        # Calculate mean survival times
        if len(male_survival) > 0 and len(female_survival) > 0:
            male_mean_days = sum(male_survival) / len(male_survival)
            female_mean_days = sum(female_survival) / len(female_survival)
            male_mean_years = male_mean_days / 365.25
            female_mean_years = female_mean_days / 365.25
            
            print(f"✅ EV-H06: TCGA-LIHC survival analysis")
            print(f"  Male mean survival: {male_mean_years:.1f} years")
            print(f"  Female mean survival: {female_mean_years:.1f} years")
            
            results["EV-H06"] = {
                "status": "success",
                "result": f"Male: {male_mean_years:.1f} years, Female: {female_mean_years:.1f} years",
                "data": {
                    "male_mean_years": round(male_mean_years, 1),
                    "female_mean_years": round(female_mean_years, 1)
                },
                "time": time.time() - start
            }
        else:
            print(f"❌ EV-H06: No survival data found")
            results["EV-H06"] = {"status": "error", "error": "No survival data", "time": time.time() - start}
            
    except Exception as e:
        print(f"❌ EV-H06 Failed: {e}")
        results["EV-H06"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_H06()

Analyzing gender survival differences in TCGA-LIHC...
✅ EV-H06: TCGA-LIHC survival analysis
  Male mean survival: 1.6 years
  Female mean survival: 2.3 years


In [ ]:
# EV-H07: In the GDC database, for TCGA-OV cases, how many have 0, 1, or multiple diagnosis records?
# Complex diagnosis record analysis with categorization
def eval_H07():
    start = time.time()
    try:
        filters = {
            "op": "=",
            "content": {"field": "project.project_id", "value": "TCGA-OV"}
        }
        
        # Get all cases with diagnoses
        result = rest_query("cases", {
            "filters": json.dumps(filters),
            "fields": "case_id,diagnoses.diagnosis_id",
            "size": "10000"
        })
        
        cases = result["data"]["hits"]
        
        # Categorize by diagnosis count
        zero_diagnoses = 0
        single_diagnosis = 0
        multiple_diagnoses = 0
        
        for case in cases:
            diagnoses = case.get("diagnoses", [])
            diagnosis_count = len(diagnoses)
            
            if diagnosis_count == 0:
                zero_diagnoses += 1
            elif diagnosis_count == 1:
                single_diagnosis += 1
            else:
                multiple_diagnoses += 1
        
        total_cases = len(cases)
        
        print(f"✅ EV-H07: TCGA-OV diagnosis record analysis")
        print(f"  Total cases: {total_cases:,}")
        print(f"  0 diagnoses: {zero_diagnoses:,} ({zero_diagnoses/total_cases*100:.1f}%)")
        print(f"  1 diagnosis:  {single_diagnosis:,} ({single_diagnosis/total_cases*100:.1f}%)")
        print(f"  Multiple:     {multiple_diagnoses:,} ({multiple_diagnoses/total_cases*100:.1f}%)")
        
        results["EV-H07"] = {
            "status": "success",
            "result": f"0: {zero_diagnoses}, 1: {single_diagnosis}, Multiple: {multiple_diagnoses}",
            "data": {
                "total_cases": total_cases,
                "zero_diagnoses": zero_diagnoses,
                "single_diagnosis": single_diagnosis,
                "multiple_diagnoses": multiple_diagnoses
            },
            "time": time.time() - start
        }
    except Exception as e:
        print(f"❌ EV-H07 Failed: {e}")
        results["EV-H07"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_H07()

✅ EV-H07: TCGA-OV diagnosis record analysis
  Total cases: 608
  0 diagnoses: 21 (3.5%)
  1 diagnosis:  216 (35.5%)
  Multiple:     371 (61.0%)


In [21]:
# EV-H08: In the GDC database, identify female breast cancer cases with WXS files, age 45-75, with survival data, ranked by project
# Most complex query: Multiple demographic + clinical + genomic filters + survival analysis + project ranking
def eval_H08():
    start = time.time()
    try:
        print("Executing complex multi-dimensional cohort analysis...")
        
        # Complex filter combining demographics, clinical, genomic, and survival data
        filters = {
            "op": "and",
            "content": [
                # Demographic filters
                {"op": "=", "content": {"field": "demographic.gender", "value": "female"}},
                {"op": ">=", "content": {"field": "diagnoses.age_at_diagnosis", "value": 16425}},  # 45 years
                {"op": "<=", "content": {"field": "diagnoses.age_at_diagnosis", "value": 27375}},  # 75 years
                
                # Clinical filters  
                {"op": "in", "content": {"field": "primary_site", "value": ["Breast"]}},
                
                # Genomic data requirement
                {"op": "=", "content": {"field": "files.experimental_strategy", "value": "WXS"}},
                
                # Survival data requirement (must have survival information)
                {"op": "not", "content": {"field": "demographic.vital_status", "value": "_missing"}}
            ]
        }
        
        # Step 1: Get cases matching all criteria
        result = rest_query("cases", {
            "filters": json.dumps(filters),
            "fields": "case_id,submitter_id,project.project_id,demographic.vital_status,demographic.days_to_death,demographic.days_to_last_follow_up,diagnoses.age_at_diagnosis",
            "size": "2000"
        })
        
        cases = result["data"]["hits"]
        total_cases = len(cases)
        
        print(f"Found {total_cases} female breast cancer cases (age 45-75) with WXS + survival data")
        
        # Step 2: Complex analysis - categorize by survival outcomes
        alive_cases = []
        deceased_cases = []
        survival_data = []
        
        for case in cases:
            demographic = case.get("demographic", {})
            vital_status = demographic.get("vital_status")
            days_to_death = demographic.get("days_to_death")
            days_to_followup = demographic.get("days_to_last_follow_up")
            
            # Get age at diagnosis
            age_at_diagnosis = None
            diagnoses = case.get("diagnoses", [])
            if diagnoses and isinstance(diagnoses, list) and len(diagnoses) > 0:
                age_at_diagnosis = diagnoses[0].get("age_at_diagnosis")
            
            case_info = {
                "case_id": case["case_id"],
                "submitter_id": case["submitter_id"],
                "project_id": case.get("project", {}).get("project_id", "Unknown"),
                "vital_status": vital_status,
                "age_at_diagnosis": age_at_diagnosis
            }
            
            if vital_status == "Alive":
                case_info["followup_days"] = days_to_followup
                alive_cases.append(case_info)
            elif vital_status == "Dead":
                case_info["survival_days"] = days_to_death
                deceased_cases.append(case_info)
                if days_to_death:
                    survival_data.append(days_to_death)
        
        # Step 3: Additional survival analysis
        # Calculate age statistics
        all_ages = []
        for case in cases:
            diagnoses = case.get("diagnoses", [])
            if diagnoses and isinstance(diagnoses, list) and len(diagnoses) > 0:
                age = diagnoses[0].get("age_at_diagnosis")
                if age:
                    all_ages.append(age / 365.25)  # Convert to years
        
        avg_age = sum(all_ages) / len(all_ages) if all_ages else 0
        
        print(f"\nSurvival Analysis:")
        print(f"    Alive: {len(alive_cases)} cases")
        print(f"    Deceased: {len(deceased_cases)} cases")
        print(f"    Average age at diagnosis: {avg_age:.1f} years")
        
        if survival_data:
            avg_survival_days = sum(survival_data) / len(survival_data)
            avg_survival_years = avg_survival_days / 365.25
            print(f"    Average survival time: {avg_survival_years:.1f} years")
            
            # Additional survival statistics
            survival_years = [days / 365.25 for days in survival_data]
            survival_years.sort()
            median_survival = survival_years[len(survival_years)//2] if survival_years else 0
            print(f"    Median survival time: {median_survival:.1f} years")
        
        results["EV-H08"] = {
            "status": "success",
            "result": f"{total_cases} female breast cancer cases (45-75y) with WXS + survival data",
            "data": {
                "total_cases": total_cases,
                "alive_cases": len(alive_cases),
                "deceased_cases": len(deceased_cases),
                "avg_age": avg_age,
                "avg_survival_years": (sum(survival_data) / len(survival_data) / 365.25) if survival_data else 0,
                "median_survival_years": (sorted([d/365.25 for d in survival_data])[len(survival_data)//2]) if survival_data else 0
            },
            "time": time.time() - start
        }
    except Exception as e:
        print(f"❌ EV-H08 Failed: {e}")
        results["EV-H08"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_H08()

Executing complex multi-dimensional cohort analysis...
Found 1033 female breast cancer cases (age 45-75) with WXS + survival data

Survival Analysis:
    Alive: 687 cases
    Deceased: 101 cases
    Average age at diagnosis: 58.7 years
    Average survival time: 4.6 years
    Median survival time: 3.5 years
Found 1033 female breast cancer cases (age 45-75) with WXS + survival data

Survival Analysis:
    Alive: 687 cases
    Deceased: 101 cases
    Average age at diagnosis: 58.7 years
    Average survival time: 4.6 years
    Median survival time: 3.5 years


In [14]:
# EV-H09: In the GDC database, find TCGA-BRCA breast cancer cases that have ALL of the following: (1) RNA-Seq experimental files, (2) complete survival status information, AND (3) AJCC pathologic stage data. Report: total cases, survival counts (alive/deceased), average survival time in years, and age group distribution (under 50, 50 to 65, over 65).
# Complex multi-entity analysis with clinical stratification
def eval_H09():
    start = time.time()
    try:
        print("Analyzing TCGA-BRCA cases with RNA-Seq, survival, and staging data...")
        
        # Define filter for cases with required data elements
        filters = {
            "op": "and",
            "content": [
                {"op": "=", "content": {"field": "project.project_id", "value": "TCGA-BRCA"}},
                {"op": "in", "content": {"field": "primary_site", "value": ["Breast"]}},
                {"op": "=", "content": {"field": "files.experimental_strategy", "value": "RNA-Seq"}},
                {"op": "not", "content": {"field": "demographic.vital_status", "value": "_missing"}},
                {"op": "not", "content": {"field": "diagnoses.ajcc_pathologic_stage", "value": "_missing"}}
            ]
        }
        
        # First get the total count
        count_result = rest_query("cases", {
            "filters": json.dumps(filters),
            "size": "0"
        })
        actual_total = count_result["data"]["pagination"]["total"]
        print(f"Total matching cases: {actual_total}")
        
        print("Fetching all cases with complete data...")
        
        # Get all cases with clinical and genomic data (increase size to capture all)
        result = rest_query("cases", {
            "filters": json.dumps(filters),
            "fields": "submitter_id,demographic.vital_status,demographic.days_to_death,diagnoses.age_at_diagnosis,diagnoses.ajcc_pathologic_stage",
            "size": str(max(2000, actual_total))  # Ensure we get all cases
        })
        
        cases = result["data"]["hits"]
        total_cases = len(cases)
        
        print(f"Found {total_cases} BRCA cases with required data")
        
        # Analyze survival and clinical characteristics
        alive_cases = 0
        deceased_cases = 0
        survival_times = []
        
        # Age group analysis (under 50, 50-65, over 65)
        age_groups = {"under_50": 0, "50_to_65": 0, "over_65": 0}
        missing_age_count = 0
        
        # Stage distribution tracking
        stage_distribution = {}
        
        for case in cases:
            # Process survival status
            demo = case.get("demographic", {})
            vital_status = demo.get("vital_status")
            days_to_death = demo.get("days_to_death")
            
            if vital_status == "Alive":
                alive_cases += 1
            elif vital_status == "Dead":
                deceased_cases += 1
                if days_to_death:
                    survival_times.append(days_to_death)
            
            # Process age and stage data
            diagnoses = case.get("diagnoses", [])
            if diagnoses:
                age_days = diagnoses[0].get("age_at_diagnosis")
                if age_days:
                    age_years = age_days / 365.25
                    if age_years < 50:
                        age_groups["under_50"] += 1
                    elif age_years <= 65:
                        age_groups["50_to_65"] += 1
                    else:
                        age_groups["over_65"] += 1
                else:
                    missing_age_count += 1
            else:
                missing_age_count += 1
                
                # Track stage distribution
                stage = diagnoses[0].get("ajcc_pathologic_stage")
                if stage and stage != "_missing":
                    stage_distribution[stage] = stage_distribution.get(stage, 0) + 1
        
        # Calculate survival statistics
        avg_survival_years = 0
        if survival_times:
            avg_survival_years = sum(survival_times) / len(survival_times) / 365.25
        
        print(f"Total cases: {total_cases}")
        print(f"Alive: {alive_cases}, Deceased: {deceased_cases}")
        
        if survival_times:
            print(f"Average survival time: {avg_survival_years:.1f} years")
        
        print(f"\nAge group distribution:")
        for age_group, count in age_groups.items():
            print(f"  {age_group.replace('_', ' ')}: {count} cases")
        
        # Show missing age data count
        age_total = sum(age_groups.values())
        print(f"  Total with age data: {age_total} cases")
        print(f"  Missing age data: {missing_age_count} cases")
        print(f"  Verification: {age_total + missing_age_count} = {total_cases}")
        
        results["EV-H09"] = {
            "status": "success",
            "result": f"{total_cases} TCGA-BRCA cases with RNA-Seq, survival, and staging data",
            "data": {
                "total_cases": total_cases,
                "alive_cases": alive_cases,
                "deceased_cases": deceased_cases,
                "avg_survival_years": round(avg_survival_years, 1),
                "age_groups": age_groups
            },
            "time": time.time() - start
        }
    except Exception as e:
        print(f"❌ EV-H09 Failed: {e}")
        results["EV-H09"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_H09()

Analyzing TCGA-BRCA cases with RNA-Seq, survival, and staging data...
Total matching cases: 1083
Fetching all cases with complete data...
Total matching cases: 1083
Fetching all cases with complete data...
Found 1083 BRCA cases with required data
Total cases: 1083
Alive: 935, Deceased: 148
Average survival time: 4.2 years

Age group distribution:
  under 50: 269 cases
  50 to 65: 438 cases
  over 65: 329 cases
  Total with age data: 1036 cases
  Missing age data: 47 cases
  Verification: 1083 = 1083
Found 1083 BRCA cases with required data
Total cases: 1083
Alive: 935, Deceased: 148
Average survival time: 4.2 years

Age group distribution:
  under 50: 269 cases
  50 to 65: 438 cases
  over 65: 329 cases
  Total with age data: 1036 cases
  Missing age data: 47 cases
  Verification: 1083 = 1083


In [17]:
# EV-H10: In the GDC database, list cases that have both WXS and RNA-Seq files
# Multi-entity intersection (cases with both WXS AND RNA-Seq)
def eval_H10():
    start = time.time()
    try:
        print("🚀 Finding cases with both WXS and RNA-Seq files...")
        print("=" * 60)
        
        # Step 1: Get cases with WXS OR RNA-Seq
        filters = {
            "op": "or",
            "content": [
                {"op": "=", "content": {"field": "files.experimental_strategy", "value": "WXS"}},
                {"op": "=", "content": {"field": "files.experimental_strategy", "value": "RNA-Seq"}}
            ]
        }
        
        # Get total count
        count_result = rest_query("cases", {
            "filters": json.dumps(filters),
            "size": "0"
        })
        total_relevant = count_result["data"]["pagination"]["total"]
        print(f"  📈 Total cases with WXS OR RNA-Seq: {total_relevant:,}")
        
        # Step 2: Process cases to find intersection
        cases_with_both = []
        wxs_cases = set()
        rnaseq_cases = set()
        
        # Fetch in batches
        size = 1000
        from_idx = 0
        
        while from_idx < total_relevant:
            result = rest_query("cases", {
                "filters": json.dumps(filters),
                "size": str(size),
                "from": str(from_idx),
                "fields": "submitter_id,case_id,files.experimental_strategy"
            })
            
            batch_cases = result["data"]["hits"]
            if not batch_cases:
                break
            
            for case in batch_cases:
                files = case.get("files", [])
                case_strategies = set()
                
                for file_info in files:
                    strategy = file_info.get("experimental_strategy")
                    if strategy in ["WXS", "RNA-Seq"]:
                        case_strategies.add(strategy)
                
                case_id = case["submitter_id"]
                if "WXS" in case_strategies:
                    wxs_cases.add(case_id)
                if "RNA-Seq" in case_strategies:
                    rnaseq_cases.add(case_id)
                
                if "WXS" in case_strategies and "RNA-Seq" in case_strategies:
                    cases_with_both.append(case_id)
            
            from_idx += len(batch_cases)
            if len(batch_cases) < size:
                break
        
        count = len(cases_with_both)
        
        print(f"\n🎯 RESULTS:")
        print(f"  Cases with WXS: {len(wxs_cases):,}")
        print(f"  Cases with RNA-Seq: {len(rnaseq_cases):,}")
        print(f"  Cases with BOTH: {count:,}")
        
        results["EV-H10"] = {
            "status": "success",
            "result": f"{count} cases with both WXS and RNA-Seq",
            "data": {
                "both_count": count,
                "wxs_count": len(wxs_cases),
                "rnaseq_count": len(rnaseq_cases)
            },
            "time": time.time() - start
        }
    except Exception as e:
        print(f"❌ EV-H10 Failed: {e}")
        results["EV-H10"] = {"status": "error", "error": str(e), "time": time.time() - start}

eval_H10()

🚀 Finding cases with both WXS and RNA-Seq files...
  📈 Total cases with WXS OR RNA-Seq: 27,151
  📈 Total cases with WXS OR RNA-Seq: 27,151

🎯 RESULTS:
  Cases with WXS: 22,354
  Cases with RNA-Seq: 22,182
  Cases with BOTH: 17,422

🎯 RESULTS:
  Cases with WXS: 22,354
  Cases with RNA-Seq: 22,182
  Cases with BOTH: 17,422
